In [ ]:
import numpy as np
import pylab
import matplotlib
import scipy
from scipy import optimize
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable



In [ ]:
CovMatrix=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Matrices/AnalysisCovariance.txt")
Nbins=len(CovMatrix)



In [ ]:
#This makes a matrix out of two gradients and a correlation coefficient
# Format: DiscreteParams[0:Nbins] = First gradient
#         DiscreteParams[Nbins:2.*Nbins] = Second gradient
#         DiscreteParams[-1] = Correlation coefficient
def MakeDiscreteMatrixTwoTerms(DiscreteParams):
    CovMatrixToReturn=numpy.ndarray(shape=(Nbins,Nbins))
    alpha=DiscreteParams[-1]
    FirstGradient=DiscreteParams[0:Nbins]
    SecondGradient=DiscreteParams[Nbins:2*Nbins]
    for i in range(0,Nbins):
        for j in range(0,Nbins):
            CovMatrixToReturn[i][j]=(FirstGradient[i]*FirstGradient[j]+ SecondGradient[i]*SecondGradient[j]+ alpha*(FirstGradient[i]*SecondGradient[j]+FirstGradient[j]*SecondGradient[i]))
    return CovMatrixToReturn

In [ ]:
#This is the function to be minimized
def MinimizeFunctionTwoTerms(DiscreteParams):
    Resid2= (MakeDiscreteMatrixTwoTerms(DiscreteParams)-CovMatrix)
    AlphaPrior=numpy.exp(100.*pow(DiscreteParams[-1],2))
    return(numpy.sum(numpy.abs(Resid2))*AlphaPrior)

In [ ]:
#These are the values we will use to seed the fit
SeedParams2=numpy.ones(2*Nbins+1)
SeedParams2[-1]=0.1

In [ ]:
#Make the fit
ResultsTwoTerms=scipy.optimize.minimize(MinimizeFunctionTwoTerms,SeedParams2)

#Get the best fit matrix
BestCov=MakeDiscreteMatrixTwoTerms(ResultsTwoTerms.x)

In [ ]:
def DrawMatrix(Matrix,SaveName="",scale=0):
    pylab.figure(figsize=(4,4),dpi=300)   
    im =pylab.imshow(Matrix,interpolation='nearest',cmap='RdBu')
    if(scale==0):
        pylab.clim(-np.max(np.abs(Matrix)),np.max(np.abs(Matrix)))
    else:
        pylab.clim(-scale,scale)
    pylab.xlabel("Energy Bin")
    pylab.ylabel("Energy Bin")


    ax=pylab.gca()
    pylab.xticks(range(len(Matrix)))
    pylab.yticks(range(len(Matrix)))
    ax.xaxis.tick_top()
    ax.xaxis.label_position='top'
    ax.xaxis.labelpad=15
    pylab.xlim(-0.5,12.5)
    pylab.ylim(12.5,-0.5)


    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    pylab.colorbar(im, cax=cax)
    pylab.tight_layout()
    if(SaveName!=""):
        pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/Plots/"+str(SaveName)+".png",dpi=250,bbox_inches='tight')

    pylab.show()




In [ ]:
DrawMatrix(BestCov,"BestFitCov",0.003)

In [ ]:
Residual=BestCov-CovMatrix
DrawMatrix(Residual,"Residual",0.003)

In [ ]:
pylab.figure(figsize=(4,4),dpi=300)

#pylab.plot(numpy.sqrt(numpy.diag(BestCov0)),label="1 Effective Gradient")
pylab.plot(numpy.sqrt(numpy.diag(BestCov)),label="2 Effective Gradients")
pylab.plot(numpy.sqrt(numpy.diag(CovMatrix)),label="Original cov matrix")
pylab.legend(loc='lower right')
pylab.ylabel("Diagonal uncertainty")
pylab.xlabel("Bin number")
pylab.tight_layout()
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/Plots/DiagonalUncertainty.png",dpi=250,bbox_inches='tight')

In [ ]:

pylab.figure(figsize=(4,4),dpi=300)

pylab.plot(ResultsTwoTerms.x[0:Nbins],label='Effective Gradient 1')
pylab.plot(ResultsTwoTerms.x[Nbins:2*Nbins],label='Effective Gradient 2')
pylab.legend(loc='upper right',title="Correlation Coeff="+str(numpy.round(ResultsTwoTerms.x[-1],decimals=3)))
pylab.xlabel("Energy Bin")
pylab.ylabel("Gradient")
pylab.tight_layout()
pylab.savefig("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/Plots/Gradients.png",dpi=250,bbox_inches='tight')

In [ ]:
np.savetxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/Gradient1.txt",ResultsTwoTerms.x[0:Nbins])
np.savetxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/Gradient2.txt",ResultsTwoTerms.x[Nbins:2*Nbins])
np.savetxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/Correlation.txt",[ResultsTwoTerms.x[-1]])

In [ ]:
EnergyAmpGrads=np.loadtxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/Gradients/FracGrad/EnergyAmpGrads.csv")
BinDefs=[EnergyAmpGrads[EnergyAmpGrads[:,0]==0][:,1],EnergyAmpGrads[EnergyAmpGrads[:,0]==0][:,2]]
np.savetxt("/data/ana/NuFSGenMC/MultiSim/MultisimOutputs/EffGradients/BinDefs",BinDefs)